### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

### read in policy data 

#### china subnational data

In [35]:
url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/China/OxCGRT_CHN_latest%20LEGACY.csv"

In [36]:
china_df = pd.read_csv(url)

In [37]:
china_df.columns = china_df.columns.str.lower().str.replace(" ", "_")

In [38]:
china_df["date"] = pd.to_datetime(china_df["date"], format="%Y%m%d")

In [41]:
china_df.sort_values("date", ascending=False)

,countryname,countrycode,regionname,regioncode,jurisdiction,date,c1_school_closing,c1_flag,c2_workplace_closing,c2_flag,c3_cancel_public_events,c3_flag,c4_restrictions_on_gatherings,c4_flag,c5_close_public_transport,c5_flag,c6_stay_at_home_requirements,c6_flag,c7_restrictions_on_internal_movement,c7_flag,c8_international_travel_controls,e1_income_support,e1_flag,e2_debt/contract_relief,e3_fiscal_measures,e4_international_support,h1_public_information_campaigns,h1_flag,h2_testing_policy,h3_contact_tracing,h4_emergency_investment_in_healthcare,h5_investment_in_vaccines,h6_facial_coverings,h6_flag,h7_vaccination_policy,h7_flag,h8_protection_of_elderly_people,h8_flag,m1_wildcard,v1_vaccine_prioritisation_(summary),v2a_vaccine_availability_(summary),v2b_vaccine_age_eligibility/availability_age_floor_(general_population_summary),v2c_vaccine_age_eligibility/availability_age_floor_(at_risk_summary),v2d_medically/_clinically_vulnerable_(non-elderly),v2e_education,v2f_frontline_workers__(non_healthcare),v2g_frontline_workers__(healthcare),v3_vaccine_financial_support_(summary),v4_mandatory_vaccination_(summary),confirmedcases,confirmeddeaths,stringencyindex,stringencyindexfordisplay,stringencylegacyindex,stringencylegacyindexfordisplay,governmentresponseindex,governmentresponseindexfordisplay,containmenthealthindex,containmenthealthindexfordisplay,economicsupportindex,economicsupportindexfordisplay
31455,China,CHN,NaN,NaN,NAT_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.61,NaN,74.52,NaN,78.65,NaN,80.95,NaN,62.5
12778,China,CHN,Heilongjiang,CHN_HL,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14744,China,CHN,Jilin,CHN_JL,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26540,China,CHN,Tianjin,CHN_TJ,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2948,China,CHN,Chongqing,CHN_CQ,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25558,China,CHN,Tianjin,CHN_TJ,STATE_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.000000e+00,0.0,0.0,NaN,0.0,0.0,0.000000e+00,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0
26541,China,CHN,Xinjiang,CHN_XJ,STATE_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,1.0,0.000000e+00,0.0,0.0,NaN,0.0,0.0,0.000000e+00,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.00,0.0,0.00,3.12,3.12,0.0,0.00,25.0,25.0
27524,China,CHN,Tibet,CHN_XZ,STATE_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,5.265491e+08,0.0,0.0,NaN,0.0,0.0,2.685249e+08,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0
28507,China,CHN,Yunnan,CHN_YN,STATE_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,N

#### just mapping china's stringency index over time (not subnational)

In [12]:
china_slim = china_df[china_df["jurisdiction"] == "NAT_TOTAL"][
    ["date", "stringencyindex"]
]

In [13]:
alt.Chart(china_slim).mark_line().encode(x="date", y="stringencyindex")

alt.Chart(...)

#### for most regions, there's no data on the most recent date--have to go back to july

In [14]:
china_df[china_df["date"] == china_df["date"].max()].head()

,countryname,countrycode,regionname,regioncode,jurisdiction,date,c1_school_closing,c1_flag,c2_workplace_closing,c2_flag,c3_cancel_public_events,c3_flag,c4_restrictions_on_gatherings,c4_flag,c5_close_public_transport,c5_flag,c6_stay_at_home_requirements,c6_flag,c7_restrictions_on_internal_movement,c7_flag,c8_international_travel_controls,e1_income_support,e1_flag,e2_debt/contract_relief,e3_fiscal_measures,e4_international_support,h1_public_information_campaigns,h1_flag,h2_testing_policy,h3_contact_tracing,h4_emergency_investment_in_healthcare,h5_investment_in_vaccines,h6_facial_coverings,h6_flag,h7_vaccination_policy,h7_flag,h8_protection_of_elderly_people,h8_flag,m1_wildcard,v1_vaccine_prioritisation_(summary),v2a_vaccine_availability_(summary),v2b_vaccine_age_eligibility/availability_age_floor_(general_population_summary),v2c_vaccine_age_eligibility/availability_age_floor_(at_risk_summary),v2d_medically/_clinically_vulnerable_(non-elderly),v2e_education,v2f_frontline_workers__(non_healthcare),v2g_frontline_workers__(healthcare),v3_vaccine_financial_support_(summary),v4_mandatory_vaccination_(summary),confirmedcases,confirmeddeaths,stringencyindex,stringencyindexfordisplay,stringencylegacyindex,stringencylegacyindexfordisplay,governmentresponseindex,governmentresponseindexfordisplay,containmenthealthindex,containmenthealthindexfordisplay,economicsupportindex,economicsupportindexfordisplay
982,China,CHN,Anhui,CHN_AH,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1965,China,CHN,Beijing,CHN_BJ,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2948,China,CHN,Chongqing,CHN_CQ,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3931,China,CHN,Fujian,CHN_FJ,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4914,China,CHN,Guangdong,CHN_GD,STATE_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
## here are the most recent dates with available data for each region
china_max_dates=china_df[(~china_df["stringencyindex"].isna())].groupby(
    "regionname", as_index=False
).agg({"date": "max"}).sort_values("date", ascending=True)

NameError: name 'china_df' is not defined

### group  by date and count the number of '3' (strict) c6_stay_at_home_requirements over time (how many regions have the strictest stay at home requirements on a given day)
last date is the day we know is the most up to date for every region

In [16]:
stayathome_overtime = china_df[
    (china_df["date"] <= "2022-07-06") & (china_df["jurisdiction"] != "NAT_TOTAL")
][["regionname", "date", "c6_stay_at_home_requirements"]]

In [17]:
stay_at_home_regions = (
    stayathome_overtime.groupby(["date", "c6_stay_at_home_requirements"])
    .size()
    .reset_index(name="count")
)

In [18]:
alt.Chart(stay_at_home_regions).mark_line().encode(
    x="date", y="count", color="c6_stay_at_home_requirements:N"
)

alt.Chart(...)

In [19]:
# plotting just the number of regions with the strictest lockdowns in place
alt.Chart(
    stay_at_home_regions[stay_at_home_regions["c6_stay_at_home_requirements"] == 3]
).mark_bar().encode(x="date", y="count", color="c6_stay_at_home_requirements:N")

alt.Chart(...)

#### grouping them by not strict (no stay at home or recommend stay at home) and very strict (stay at home unless essential or stay at home period)

In [20]:
conditions = [
    (
        (stayathome_overtime["c6_stay_at_home_requirements"] == 0)
        | (stayathome_overtime["c6_stay_at_home_requirements"] == 1)
    ),
    (
        (stayathome_overtime["c6_stay_at_home_requirements"] == 2)
        | (stayathome_overtime["c6_stay_at_home_requirements"] == 3)
    ),
]
values = [
    "stay at home recommended or not required",
    "strict or essential stay at home",
]

stayathome_overtime["stayathome_categories"] = np.select(conditions, values)

In [21]:
stayathome_text = (
    stayathome_overtime.groupby(["date", "stayathome_categories"])
    .size()
    .reset_index(name="count")
)

In [22]:
alt.Chart(stayathome_text).mark_line().encode(
    x="date", y="count", color="stayathome_categories:N"
)

alt.Chart(...)

#### getting the regions by stringency index over time

In [23]:
# 7/6 is the most recent date for which every region has data
regions_overtime = china_df[
    (china_df["date"] <= "2022-07-06") & (china_df["jurisdiction"] != "NAT_TOTAL")
][["regionname", "date", "stringencyindex"]]

In [24]:
regions_stringency = regions_overtime[["regionname", "date", "stringencyindex"]]

#### the dataset is too big to plot every region, so just starting with a few big ones

In [25]:
regions = [
    "Sichuan",
    "Shanghai",
    "Beijing",
    "Hunan",
]

In [26]:
# go with the regions that have the most up to date data--it was only 12 but still too big to plot
date_df = (
    china_df[(~china_df["stringencyindex"].isna())]
    .groupby("regionname", as_index=False)
    .agg({"date": "max"})
    .sort_values("date", ascending=True)
)
regions = pd.Series(date_df[date_df["date"] == date_df["date"].max()]["regionname"])

In [27]:
alt.Chart(
    regions_stringency[regions_stringency["regionname"].isin(regions)]
).mark_line().encode(x="date", y="stringencyindex", color="regionname:N")

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.Chart(...)

### international indices

In [42]:
url = "https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_nat_latest.csv"

In [43]:
df = pd.read_csv(url)

In [44]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [45]:
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

In [47]:
df.sort_values("date", ascending=False)

,countryname,countrycode,regionname,regioncode,jurisdiction,date,c1m_school_closing,c1m_flag,c2m_workplace_closing,c2m_flag,c3m_cancel_public_events,c3m_flag,c4m_restrictions_on_gatherings,c4m_flag,c5m_close_public_transport,c5m_flag,c6m_stay_at_home_requirements,c6m_flag,c7m_restrictions_on_internal_movement,c7m_flag,c8ev_international_travel_controls,e1_income_support,e1_flag,e2_debt/contract_relief,e3_fiscal_measures,e4_international_support,h1_public_information_campaigns,h1_flag,h2_testing_policy,h3_contact_tracing,h4_emergency_investment_in_healthcare,h5_investment_in_vaccines,h6m_facial_coverings,h6m_flag,h7_vaccination_policy,h7_flag,h8m_protection_of_elderly_people,h8m_flag,m1_wildcard,v1_vaccine_prioritisation_(summary),v2a_vaccine_availability_(summary),v2b_vaccine_age_eligibility/availability_age_floor_(general_population_summary),v2c_vaccine_age_eligibility/availability_age_floor_(at_risk_summary),v2d_medically/_clinically_vulnerable_(non-elderly),v2e_education,v2f_frontline_workers__(non_healthcare),v2g_frontline_workers__(healthcare),v3_vaccine_financial_support_(summary),v4_mandatory_vaccination_(summary),confirmedcases,confirmeddeaths,majorityvaccinated,populationvaccinated,stringencyindex_average,stringencyindex_average_fordisplay,governmentresponseindex_average,governmentresponseindex_average_fordisplay,containmenthealthindex_average,containmenthealthindex_average_fordisplay,economicsupportindex,economicsupportindex_fordisplay
183820,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.61,NaN,49.91,NaN,57.04,NaN,0.0
52098,Eritrea,ERI,NaN,NaN,NAT_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86503,Japan,JPN,NaN,NaN,NAT_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.53,NaN,49.75,NaN,42.57,NaN,100.0
97316,Libya,LBY,NaN,NaN,NAT_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.74,NaN,38.72,NaN,44.26,NaN,0.0
9829,Azerbaijan,AZE,NaN,NaN,NAT_TOTAL,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.69,NaN,37.64,NaN,43.01,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140569,Rwanda,RWA,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NV,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
44235,Djibouti,DJI,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NV,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
139586,Russia,RUS,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NV,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
45218,Dominica,DMA,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,N

### figure out how to do a groupby that instead of aggregating just maintains certain columns

Goal: Remove rows with missing data, group by country, within each country, select the most recent date (that has data, hence removing missing data), and show the existing data for those rows<br>
I can't figure out how to groupby and select the most recent date and still keep the datapoints--when I include the datapoints in the agg, is there a way to say, don't do any function?<r>
In R it looked like this:<br>
people_vaccinated <- vaccines_by_country_raw %>%<br>
  select(iso_code, location, date, people_vaccinated) %>%<br> #### retains the data columns I want without any aggregating or summarizing function<br>
  filter(!is.na(people_vaccinated)) %>%<br> #### gets rid of rows w missing data<br>
  group_by(iso_code, location) %>%<br> #### groups by country<br>
  slice(which.max(date))<br>  #### within that grouping takes the most recent date<br><br><br>



In [63]:
max_dates = (
    df[(~df["stringencyindex_average"].isna())]
    .groupby("countryname", as_index=False)
    .agg({"date": "max"})
)

In [64]:
merged = pd.merge(
    df[["countryname", "date", "stringencyindex_average"]],
    max_dates,
    on=["countryname", "date"],
)